In [1]:
from models.from_chatgpt import Transformer
from utils.utils import print_model_memory_footprint
from datalib import download_wiki_sample

In [2]:
model = Transformer()
print_model_memory_footprint(model)

The model has 24,561,424 parameters.
The model has 0 tensors.
The model's memory footprint is approximately 93.69 MB.


In [2]:
train_data, eval_data = download_wiki_sample.download_and_split_wiki_dataset()

train[0:100000]+train[-1000:]


Found cached dataset wikipedia (/Users/skorodumov-s/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


In [3]:
print(len(train_data))
print(len(eval_data))

100000
1000


In [4]:
from torch.utils.data import DataLoader
from transformers import AdamW
from utils import send_metrics

np.random.seed(42) # TODO check if it actually makes shuffle below consistent
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
eval_dataloader = DataLoader(eval_data, batch_size=32)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=0.001)

# Start training
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(inputs, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in eval_dataloader:
            inputs = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(inputs, labels=labels)
            total_eval_loss += outputs.loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    avg_eval_loss = total_eval_loss / len(eval_dataloader)

    # Send metrics
    send_metrics(avg_train_loss, avg_eval_loss)


ModuleNotFoundError: No module named 'transformers'